In [1]:
import tweepy
import pandas
import csv
import re #regular expression
import nltk
from nltk.corpus import stopwords
##nltk.download('stopwords')# Download stopwords
from textblob import TextBlob
import sys
import re #regular expression
import psycopg2
import time
import numpy as np

In [2]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

In [3]:
consumer_key = '1QRIUdf4Sx4xdMDc46nQ6vYZ3'
consumer_secret = '5tEVf8mm8dUjYftXiae9QERBDFiKXrVSWkjRm2jpfBNy0NRm1Q'
access_token = '1191719379186192384-Gn0pBMspYGX2BCQxqyZACjNtAQ67YL'
access_secret = 'WuVwxRsXdw9gtTK1OTWK7McCtAC7l8Y0BSfYEUhcYEWv0'
access_token_secret=access_secret

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True )

non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
##print(x.translate(non_bmp_map))

In [4]:
def cleaning (text):
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    #removing emoji
    text = emoji_pattern.sub(r' ', text)   

    #removeing http and https (URL)
    text = re.sub(r'(http://|https://)\S+', '', text)
    
    #removing www (URL)
    text=re.sub(r'www\.\S+', '', text)
    
    #removing targets
    text=re.sub('( |^)@\S+', '', text) 
    
    #removing tabs and lines
    text=re.sub('\t|\n', ' ', text)

    #removing some special charachter  
    text= re.sub("[\"\“\”\+\-\|\*\?\(\)\/\\\^\[\]\.{}_`;•«,@:~!=%&]+", ' ',text) # except ' ’
    
    #removing hashtag
    text=re.sub('#', '', text) 
    
    #removing numbers
    text=re.sub("(\d+)",' ',text)
#    print(text)
    #removing numbers (not attached)
#    text=re.sub(r"(^|\s)-?(\d+)\.?(\d*)(\s)", '', text)
#    text=re.sub("(^||\d+)\.(\d+) ",'',text)
#    text=re.sub("[0-9+]\.?(\d+)(\.)",' ',text)
#    text=re.sub("[0-9+]\.?(\d+)($)",' ',text)
#    print(text)
    
#    return text

#def lower_case (text):
    text = text.lower() 
#    print(text)
#    return text_lower

#def tokenization (text):
    text= nltk.word_tokenize(text)
#    print (text)
#    return text_tokens

#def removing_stopwords (text):
    #text_without_sw = [word.lower() for word in text if word.lower() not in stopwords.words()]
#    text = [word for word in text if word not in stopwords.words()]
    text = [word for word in text if word not in cachedStopWords]
#    print(text)
#    return text_without_sw

  
    text=' '.join(text)
#    print(text)
#    print(text,'\n')   
    
    return text

In [6]:
search_term = "* "
##search_term = "'#entrepreneur' OR business owner'' OR 'founded' OR 'founder'  -filter:retweets"
tweets = tweepy.Cursor(api.search,
                       q=search_term,
                       result_type="recent",
                       lang="en",
                       tweet_mode="extended",
                       ).items() 

In [7]:
KEYWORDS_list=['manager','entrepreneur','management','founder','founded' ,'business owner']
n_total=0

In [ ]:
# Collecting target tweets
for tweet in tweets:

##    print(tweet.full_text.translate(non_bmp_map))
##    print(tweet.user.screen_name.translate(non_bmp_map))
##    print()

##    #Create a sublist of lower case words for each tweet
##    words_in_tweet = [tweet.full_tetx.lower().split() for tweet in tweets]

    words_in_description=cleaning(tweet.user.description)
##    key__words=['entrepreneur','founder','founded','business owner']
##    if set(key__words) & set(words_in_description.split(' ')):

    Istarget=1
    for KEY in KEYWORDS_list:
        if KEY in words_in_description:
            Istarget=0
    if Istarget==1:
        n_total +=1
            
        data1= [tweet.user.screen_name.translate(non_bmp_map),
               tweet.user.name.translate(non_bmp_map),
               tweet.user.id,
               tweet.full_text.translate(non_bmp_map),
               cleaning(tweet.full_text),
               tweet.lang,
               tweet.created_at,
               tweet.user.location.translate(non_bmp_map),
               tweet.user.description,
               str(tweet._json)]

        conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=Jafarsql")
        cur = conn.cursor()
        cur.execute("INSERT INTO public_world_tweets_ph2 VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",data1);
        conn.commit()

Rate limit reached. Sleeping for: 623
Rate limit reached. Sleeping for: 649
Rate limit reached. Sleeping for: 732
